# Epilepsy Detection with Deep Learning (LSTM)

This notebook demonstrates how to train an LSTM-based deep learning model for EEG-based epilepsy detection.
It follows the same dataset structure as your previous notebooks: 178 features per sample + label column.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
print('TensorFlow version:', tf.__version__)

: 

In [3]:
# Load dataset (expects 178 features + 'y' or last column as label)

df = pd.read_csv('data.csv')
for col in list(df.columns):
    if 'unnamed' in col.lower():
        df = df.drop(columns=[col])

label_col = 'y' if 'y' in df.columns else df.columns[-1]
X = df.drop(columns=[label_col]).values
y_raw = df[label_col].values

# Convert to binary: 1 → epileptic, else non-epileptic
y = (y_raw == 1).astype(int)

print('Feature shape:', X.shape, 'Label shape:', y.shape)
print('Class distribution:', {0:int((y==0).sum()), 1:int((y==1).sum())})

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
# Preprocessing: scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Reshape to (samples, timesteps, channels)
X_train_seq = X_train.reshape((-1, 178, 1))
X_test_seq = X_test.reshape((-1, 178, 1))
print('Reshaped train:', X_train_seq.shape, 'Reshaped test:', X_test_seq.shape)

In [ ]:
# Build LSTM model
model = keras.Sequential([
    layers.Input(shape=(178,1)),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]

history = model.fit(X_train_seq, y_train,
                    validation_split=0.2,
                    epochs=30,
                    batch_size=64,
                    callbacks=callbacks)

In [ ]:
# Plot training history
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')

plt.show()

In [ ]:
# Evaluate
loss, acc = model.evaluate(X_test_seq, y_test, verbose=0)
print(f'Test loss: {loss:.4f}, Test accuracy: {acc:.4f}')

# Predictions
y_pred_prob = model.predict(X_test_seq).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

In [ ]:
# Confusion Matrix Heatmap
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non-Epileptic', 'Epileptic'],
            yticklabels=['Non-Epileptic', 'Epileptic'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Save model and scaler
model.save('deeplearning_model')
import joblib
joblib.dump(scaler, '/mnt/data/lstm_scaler.save')
print('Saved model to /mnt/data/lstm_epilepsy_model and scaler to /mnt/data/lstm_scaler.save')